In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as spst

#import sys
#sys.path.insert(sys.path.index('')+1, 'D:/Github/pyfeng')
import pyfeng as pf

# Garch Diffusion Model

Barone-Adesi G, Rasmussen H, Ravanelli C (2005) An option pricing formula for the GARCH diffusion model. Computational Statistics & Data Analysis 49:287–310. https://doi.org/10.1016/j.csda.2004.05.014

### The implementations are based on the following student course projects

https://github.com/PyFE/pyfedev/blob/master/test/Test_Garch.ipynb

https://github.com/PanyuLi/PHBS_ASP_Project/blob/main/Test_CondMC_OUSV_Garch.ipynb

## Table 1

In [3]:
mr, vov, sigma = 18, 1.8, 0.16/18

strike = np.arange(90,111,5) # strike price
spot = 100 # spot asset price
texp = np.array([30, 60, 90, 120, 180, 252, 504])/252  # time to maturity

p_grid = [
    [0.0008, 0.0184, 0.0664, 0.1393, 0.3294, 0.5928, 1.5438],
    [0.0800, 0.2993, 0.5401, 0.7760, 1.2161, 1.6919, 3.0577],
    [1.2921, 1.8284, 2.2411, 2.5890, 3.1725, 3.7550, 5.3116],
    [5.0999, 5.3497, 5.6155, 5.8721, 6.3462, 6.8550, 8.3052],
    [10.0023, 10.0363, 10.1131, 10.2187, 10.4733, 10.8062, 11.9443]
]
p_paper = pd.DataFrame(data=p_grid, columns=np.int32(texp*252), index=strike)

## Table 2

In [4]:
mr, vov, sigma = 29.23, 3.65, 0.53/29.23

strike = np.arange(90,111,5) # strike price
spot = 100 # spot asset price
texp = np.array([30, 60, 90, 120, 180, 252, 504])/252  # time to maturity

p_grid = [
    [0.0236, 0.1508, 0.3449, 0.5668, 1.0310, 1.5777, 3.2715],
    [0.3093, 0.7899, 1.2348, 1.6394, 2.3550, 3.0980, 5.1528],
    [1.8344, 2.6014, 3.1915, 3.6891, 4.5229, 5.3549, 7.5754],
    [5.3602, 5.8867, 6.3659, 6.7987, 7.5601, 8.3478, 10.5187],
    [10.0436, 10.2325, 10.4914, 10.7723, 11.3372, 11.9831, 13.9310]
]
p_paper = pd.DataFrame(data=p_grid, columns=np.int32(texp*252), index=strike)

## Table 3

In [5]:
mr, vov, sigma = 2, 0.8, 0.18/2

strike = np.arange(90,111,5) # strike price
spot = 100 # spot asset price
texp = np.array([30, 60, 90, 120, 180, 252, 504])/252  # time to maturity

p_grid = [
    [0.7901, 1.9061, 2.8985, 3.7858, 5.3378, 6.9387, 11.3404],
    [2.0011, 3.5125, 4.7109, 5.7337, 7.4645, 9.2060, 13.8865],
    [4.1169, 5.8083, 7.1015, 8.1892, 10.0110, 11.8297, 16.6835],
    [7.1841, 8.7862, 10.0522, 11.1311, 12.9552, 14.7891, 19.7148],
    [11.0473, 12.3644, 13.5044, 14.5125, 16.2621, 18.0562, 22.9623],
]
p_paper = pd.DataFrame(data=p_grid, columns=np.int32(texp*252), index=strike)

## PyFeng Implementation

In [6]:
m1 = pf.GarchUncorrBaroneAdesi2004(sigma, vov=vov, mr=mr, theta=sigma)
p_grid = m1.price(strike, spot, texp[:, None], cp=-1)

p_pyfeng = pd.DataFrame(data=p_grid.T, columns=np.int32(texp*252), index=strike)

In [7]:
# Difference between the paper and pyfeng values
p_pyfeng.round(4) - p_paper

,30,60,90,120,180,252,504
90,0.0001,0.0009,0.0015,0.0020,0.0027,0.0021,0.0028
95,0.0001,0.0004,0.0001,-0.0002,-0.0009,-0.0028,-0.0034
100,-0.0005,-0.0009,-0.0021,-0.0033,-0.0053,-0.0083,-0.0099
105,-0.0008,-0.0019,-0.0039,-0.0057,-0.0090,-0.0130,-0.0162
110,-0.0001,-0.0016,-0.0040,-0.0064,-0.0108,-0.0161,-0.0214


## Monte-Carlo with Milstein Scheme 

In [8]:
m2 = pf.GarchMcTimeStep(sigma, vov=vov, mr=mr, theta=sigma)
m2.set_num_params(n_path=1e5, dt=1/50, rn_seed=123456)  #, scheme=1

## For Table 1 and 3, dt = 1/50 is enough
## For Table 2, dt = 1/100 to avoid zero v_t

In [9]:
p_grid = np.zeros((len(texp), len(strike)))
for i, t1 in enumerate(texp):
    p_grid[i, :] = m2.price(strike, spot, t1, cp=-1)
p_mc = pd.DataFrame(data=p_grid.T, columns=np.int32(texp*252), index=strike)

In [10]:
# Difference between the paper and MC method values
p_mc.round(4) - p_paper

,30,60,90,120,180,252,504
90,0.0002,-0.0001,-0.0013,-0.0014,-0.0012,0.0019,0.0033
95,-0.0004,-0.0006,-0.0019,-0.0018,-0.0015,0.0018,0.0034
100,-0.0007,-0.0008,-0.0021,-0.0020,-0.0016,0.0018,0.0035
105,-0.0005,-0.0006,-0.0020,-0.0019,-0.0016,0.0019,0.0036
110,0.0001,-0.0002,-0.0016,-0.0016,-0.0014,0.0020,0.0037
